### Zukhra Mazhitova

There is a dataset with more than 6000 victims of police officers' shootings in USA from 2015 to today's day. The task is about predicting the race of the victim, specifiaclly if he/she is black or not

### Data description


The Washinghton Post's data is mainly based on news accounts, social media postings and police reports. Review contains data of about 6,000 cases in last six years.

Chosen dataset of The Washington Post contains records of every fatal shooting in the United States by a police officer 

+ id: a unique identifiying number for each victim
+ name: the name of the killed person
+ date: the date of the shooting occured in YYYY-MM-DD format
+ manner_of_death: how the victim was
+ armed:  whether victom had a some kind of gun or potentially threataning equipment, if so what king of arm.
+ age: the age of the victim
+ gender: the biological gender of the victim. 
+ race: the racial information 
+ city: the place where the shooting occured. 
+ state: two-letter postal code abbreviation of the state 
+ signs of mental illness:  if victim had a history of mental health concerns, suicidal thoughts or suffered from  psychiatric illness.
+ threat_level: The threat_level column was used to flag incidents for the story by Amy Brittain in October 2015. 
+ flee: whether the victim was moving away from officers, if so how
+ body_camera: if an officer was wearing a body camera which may contain some kind of information about the occured incident incident.
+ latitude and longitude: the location of the shooting expressed as WGS84 coordinates, geocoded from addresses. The coordinates are rounded to 3 decimal places, meaning they have a precision of about 80-100 meters within the contiguous U.S.
+ is_geocoding_exact: reflects the accuracy of the coordinates. true means that the coordinates are for the location of the shooting (within approximately 100 meters), while false means that coordinates are for the centroid of a larger region, such as the city or county where the shooting happened.

In [1]:
#importing libraries
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer
from sklearn.pipeline import Pipeline
import category_encoders as ce

import time
import pandas as pd
import numpy as np
import us
import requests
from bs4 import BeautifulSoup
import re

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('shooting.csv', parse_dates=['date'])
df.head()

,Unnamed: 0,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera,longitude,latitude,is_geocoding_exact
0,0,3.0,Tim Elliot,2015-01-02,shot,gun,53.0,M,A,Shelton,WA,True,attack,Not fleeing,False,-123.122,47.247,True
1,1,4.0,Lewis Lee Lembke,2015-01-02,shot,gun,47.0,M,W,Aloha,OR,False,attack,Not fleeing,False,-122.892,45.487,True
2,2,5.0,John Paul Quintero,2015-01-03,shot and Tasered,unarmed,23.0,M,H,Wichita,KS,False,other,Not fleeing,False,-97.281,37.695,True
3,3,8.0,Matthew Hoffman,2015-01-04,shot,toy weapon,32.0,M,W,San Francisco,CA,True,attack,Not fleeing,False,-122.422,37.763,True
4,4,9.0,Michael Rodriguez,2015-01-04,shot,nail gun,39.0,M,H,Evans,CO,False,attack,Not fleeing,False,-104.692,40.384,True


In [3]:
df.isna().sum()

Unnamed: 0                   0
id                          27
name                       257
date                        27
manner_of_death             27
armed                      237
age                        302
gender                      28
race                       649
city                        27
state                       27
signs_of_mental_illness     27
threat_level                27
flee                       368
body_camera                 27
longitude                   27
latitude                    27
is_geocoding_exact          27
dtype: int64

In [4]:
df.shape

(6032, 18)

### Cleaning

In [5]:
new = df.drop(
    ['Unnamed: 0','id','name','date','city','flee','armed','is_geocoding_exact'],axis=1)
new.head()

,manner_of_death,age,gender,race,state,signs_of_mental_illness,threat_level,body_camera,longitude,latitude
0,shot,53.0,M,A,WA,True,attack,False,-123.122,47.247
1,shot,47.0,M,W,OR,False,attack,False,-122.892,45.487
2,shot and Tasered,23.0,M,H,KS,False,other,False,-97.281,37.695
3,shot,32.0,M,W,CA,True,attack,False,-122.422,37.763
4,shot,39.0,M,H,CO,False,attack,False,-104.692,40.384


In [6]:
new = new.dropna(subset=['race'],how='any')
new

,manner_of_death,age,gender,race,state,signs_of_mental_illness,threat_level,body_camera,longitude,latitude
0,shot,53.0,M,A,WA,True,attack,False,-123.122,47.247
1,shot,47.0,M,W,OR,False,attack,False,-122.892,45.487
2,shot and Tasered,23.0,M,H,KS,False,other,False,-97.281,37.695
3,shot,32.0,M,W,CA,True,attack,False,-122.422,37.763
4,shot,39.0,M,H,CO,False,attack,False,-104.692,40.384
...,...,...,...,...,...,...,...,...,...,...
6013,shot,52.0,M,W,MD,False,attack,False,-77.121,39.189
6018,shot,25.0,M,B,OH,False,other,False,-84.296,39.818
6023,shot,18.0,M,B,AL,False,attack,False,-88.212,30.689
6024,shot,34.0,M,B,MD,False,attack,False,-76.645,39.304


In [7]:
new.isna().sum()

manner_of_death             0
age                        97
gender                      1
race                        0
state                       0
signs_of_mental_illness     0
threat_level                0
body_camera                 0
longitude                   0
latitude                    0
dtype: int64

In [8]:
new

,manner_of_death,age,gender,race,state,signs_of_mental_illness,threat_level,body_camera,longitude,latitude
0,shot,53.0,M,A,WA,True,attack,False,-123.122,47.247
1,shot,47.0,M,W,OR,False,attack,False,-122.892,45.487
2,shot and Tasered,23.0,M,H,KS,False,other,False,-97.281,37.695
3,shot,32.0,M,W,CA,True,attack,False,-122.422,37.763
4,shot,39.0,M,H,CO,False,attack,False,-104.692,40.384
...,...,...,...,...,...,...,...,...,...,...
6013,shot,52.0,M,W,MD,False,attack,False,-77.121,39.189
6018,shot,25.0,M,B,OH,False,other,False,-84.296,39.818
6023,shot,18.0,M,B,AL,False,attack,False,-88.212,30.689
6024,shot,34.0,M,B,MD,False,attack,False,-76.645,39.304


In [9]:
new.isna().sum()

manner_of_death             0
age                        97
gender                      1
race                        0
state                       0
signs_of_mental_illness     0
threat_level                0
body_camera                 0
longitude                   0
latitude                    0
dtype: int64

In [10]:

from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values=np.NaN, strategy='most_frequent')

new.gender = imputer.fit_transform(new['gender'].values.reshape(-1,1))[:,0]

new.isna().sum()

manner_of_death             0
age                        97
gender                      0
race                        0
state                       0
signs_of_mental_illness     0
threat_level                0
body_camera                 0
longitude                   0
latitude                    0
dtype: int64

### Missing values

In [11]:
for col_name in new.columns:
    if new[col_name].dtypes == 'object':
        unique_cat = len(new[col_name].unique())
        print("Column '{col_name}' has {unique_cat} unique values".format(
            col_name=col_name, unique_cat=unique_cat
        ))

Column 'manner_of_death' has 2 unique values
Column 'gender' has 2 unique values
Column 'race' has 6 unique values
Column 'state' has 51 unique values
Column 'signs_of_mental_illness' has 2 unique values
Column 'threat_level' has 3 unique values
Column 'body_camera' has 2 unique values


In [12]:
print(new['threat_level'].value_counts().sort_values(ascending=False))

attack          3478
other           1675
undetermined     230
Name: threat_level, dtype: int64


In [13]:
X = new.drop('race', axis=1)
y = new['race']

In [14]:
X.head()

,manner_of_death,age,gender,state,signs_of_mental_illness,threat_level,body_camera,longitude,latitude
0,shot,53.0,M,WA,True,attack,False,-123.122,47.247
1,shot,47.0,M,OR,False,attack,False,-122.892,45.487
2,shot and Tasered,23.0,M,KS,False,other,False,-97.281,37.695
3,shot,32.0,M,CA,True,attack,False,-122.422,37.763
4,shot,39.0,M,CO,False,attack,False,-104.692,40.384


In [15]:
X['signs_of_mental_illness'] = [ 0 if x == False
                             else 1 if x == True
                            else 0
                            for x in X['signs_of_mental_illness']]
X['body_camera'] = [ 0 if x == False
                else 1 if x == True
                else 0
                for x in X['body_camera']]
X.head()

,manner_of_death,age,gender,state,signs_of_mental_illness,threat_level,body_camera,longitude,latitude
0,shot,53.0,M,WA,1,attack,0,-123.122,47.247
1,shot,47.0,M,OR,0,attack,0,-122.892,45.487
2,shot and Tasered,23.0,M,KS,0,other,0,-97.281,37.695
3,shot,32.0,M,CA,1,attack,0,-122.422,37.763
4,shot,39.0,M,CO,0,attack,0,-104.692,40.384


In [16]:
enc = OrdinalEncoder()
X[['state']] = enc.fit_transform(X[['state']])


In [17]:
def toDummy(df, dummy_column):
    for x in dummy_column:
        dummies=pd.get_dummies(df[x],prefix=x,dummy_na=False)
        df = df.drop(x,1)
        df = pd.concat([df, dummies], axis=1)
    return df

In [18]:
dummy_column=['manner_of_death', 'threat_level','gender']

In [19]:
X = toDummy(X, dummy_column)
print(X.head(5))

    age  state  signs_of_mental_illness  body_camera  longitude  latitude  \
0  53.0   47.0                        1            0   -123.122    47.247   
1  47.0   37.0                        0            0   -122.892    45.487   
2  23.0   16.0                        0            0    -97.281    37.695   
3  32.0    4.0                        1            0   -122.422    37.763   
4  39.0    5.0                        0            0   -104.692    40.384   

   manner_of_death_shot  manner_of_death_shot and Tasered  \
0                     1                                 0   
1                     1                                 0   
2                     0                                 1   
3                     1                                 0   
4                     1                                 0   

   threat_level_attack  threat_level_other  threat_level_undetermined  \
0                    1                   0                          0   
1                    1  

In [20]:
X.isna().sum()

age                                 97
state                                0
signs_of_mental_illness              0
body_camera                          0
longitude                            0
latitude                             0
manner_of_death_shot                 0
manner_of_death_shot and Tasered     0
threat_level_attack                  0
threat_level_other                   0
threat_level_undetermined            0
gender_F                             0
gender_M                             0
dtype: int64

### Missing Values: Iterative Imputer

In [21]:
impute_it = IterativeImputer()
new_X = pd.DataFrame(impute_it.fit_transform(X), columns=X.columns)
new_X

,age,state,signs_of_mental_illness,body_camera,longitude,latitude,manner_of_death_shot,manner_of_death_shot and Tasered,threat_level_attack,threat_level_other,threat_level_undetermined,gender_F,gender_M
0,53.0,47.0,1.0,0.0,-123.122,47.247,1.0,0.0,1.0,0.0,0.0,0.0,1.0
1,47.0,37.0,0.0,0.0,-122.892,45.487,1.0,0.0,1.0,0.0,0.0,0.0,1.0
2,23.0,16.0,0.0,0.0,-97.281,37.695,0.0,1.0,0.0,1.0,0.0,0.0,1.0
3,32.0,4.0,1.0,0.0,-122.422,37.763,1.0,0.0,1.0,0.0,0.0,0.0,1.0
4,39.0,5.0,0.0,0.0,-104.692,40.384,1.0,0.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5378,52.0,20.0,0.0,0.0,-77.121,39.189,1.0,0.0,1.0,0.0,0.0,0.0,1.0
5379,25.0,35.0,0.0,0.0,-84.296,39.818,1.0,0.0,0.0,1.0,0.0,0.0,1.0
5380,18.0,1.0,0.0,0.0,-88.212,30.689,1.0,0.0,1.0,0.0,0.0,0.0,1.0
5381,34.0,20.0,0.0,0.0,-76.645,39.304,1.0,0.0,1.0,0.0,0.0,0.0,1.0


### Missing Values: KNN Imputer

In [22]:
impute_knn = KNNImputer()
new2_X = pd.DataFrame(impute_knn.fit_transform(X), columns=X.columns)
new2_X.isna().sum()

age                                 0
state                               0
signs_of_mental_illness             0
body_camera                         0
longitude                           0
latitude                            0
manner_of_death_shot                0
manner_of_death_shot and Tasered    0
threat_level_attack                 0
threat_level_other                  0
threat_level_undetermined           0
gender_F                            0
gender_M                            0
dtype: int64

In [23]:
y 

0       A
1       W
2       H
3       W
4       H
       ..
6013    W
6018    B
6023    B
6024    B
6029    B
Name: race, Length: 5383, dtype: object

In [24]:
y.dropna()
y = [ 1 if x == 'B'
                  else 0
                 
                  for x in  y]


### Building a model

In [37]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(new_X, y,train_size=0.8)

In [38]:
import sklearn.feature_selection

select = sklearn.feature_selection.SelectKBest(k=10)
selected_features = select.fit(X_train, y_train)
indices_selected = selected_features.get_support(indices=True)
colnames_selected = [new_X.columns[i] for i in indices_selected]

X_train_selected = X_train[colnames_selected]
X_test_selected = X_test[colnames_selected]

In [39]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

def find_model_perf(X_train, y_train, X_test, y_test):
    model = LogisticRegression()
    model.fit(X_train, y_train)
    y_hat = [x[1] for x in model.predict_proba(X_test)]
    auc = roc_auc_score(y_test, y_hat)
    
    return auc

In [40]:
auc_processed = find_model_perf(X_train, y_train, X_test, y_test)
print(auc_processed)

0.7425784563189143


C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [43]:
from sklearn.model_selection import train_test_split

X_train2, X_test2, y_train2, y_test2 = train_test_split(new2_X, y,train_size=0.8)

In [44]:
auc_processed = find_model_perf(X_train2, y_train2, X_test2, y_test2)
print(auc_processed)

0.7080696640407357


C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [45]:
new.dropna(inplace=True)

X_without = new
y_without = new['race']

for col_name in new.columns:
    if new[col_name].dtypes not in ['int32','int64','float32','float64']:
        X_without = X_without.drop(col_name, 1)


In [46]:
y_without = [ 1 if x == 'B'
                  else 0
                  for x in  y_without]

In [47]:
X_without.isna().sum()

age          0
longitude    0
latitude     0
dtype: int64

In [48]:
X_train_without, X_test_without, y_train_without, y_test_without = train_test_split(
    X_without, y_without, train_size=0.8)
auc_unprocessed = find_model_perf(X_train_without, y_train_without, X_test_without, y_test_without)
print(auc_unprocessed)

0.6990559122109425


### Conclusion

With preprosessing and filling the missing values auc score was a bit higher, than without. Gender missing values where filled with Simple Imputer and most frequent value technique. Others with Iterative and KNNImputer. Iterative Imputer was more effective with 0.742 value and KNN Imputer 0.708 value. However, without any preprocessing the score is 0.699. 